## Neural Networks

This Notebook uses the `all_morphtable` data and sets up separate Neural Networks to decompose nouns, adjectives, verbs, determiners and numbers.

These Networks are already save separately and these codeblocks don't need to be reran for them to appear. Especially since some of the runtimes took close to a day (on a dual core 2.80 GHz AMD Phenom II X4 925 CPU).

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import random
random_state = 777

Using TensorFlow backend.


In [ ]:
nouns = pd.read_csv('nouns3.csv')

In [ ]:
nouns

In [ ]:
def keep_noun_row(row):
    if 'kuty' in row['word'] or 'nap' in row['word']:
        return False
    if sum(row[2:]) == 0:
        if random.random() > 0.9:
            return True
        else:
            return False
    else:
        return True

In [16]:
'''
no_morpheme_rows = 0
for i, row in nouns.iterrows():
    if sum(row[2:]) == 0:
        no_morpheme_rows += 1
no_morpheme_rows
'''

1263596

In [ ]:
nouns['keep'] = nouns.apply(lambda row: keep_noun_row(row), axis=1)

In [24]:
nouns = nouns[nouns['keep'] == True]
nouns = nouns[nouns.columns.difference(['keep'])]

In [25]:
nouns

,ANP,ANP<PLUR>,CAS<ABL>,CAS<ACC>,CAS<ADE>,CAS<ALL>,CAS<CAU>,CAS<DAT>,CAS<DEL>,CAS<ELA>,...,POSTP<SZERINT>,POSTP<SZÁMÁRA>,POSTP<UTÁN>,POSTP<VÉGBŐL>,POSTP<VÉGETT>,POSTP<VÉGRE>,POSTP<ÁLTAL>,POSTP<ÓTA>,stem,word
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,aa-láv,aa-lávaként
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,aaaz,aaazt
31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,aaaaz,aaaaz
33,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,aaah.mos,aaah.most
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,aaakar,aaakarok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3942508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,üórá,üórán
3942510,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,üögy,üögye
3942511,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,üögy,üögyében
3942512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,üönképzőkör,üönképzőkör


In [4]:
adjectives = pd.read_csv('adjectives3.csv')

In [5]:
adjectives

,word,stem,ANP,ANP<PLUR>,CAS<ABL>,CAS<ACC>,CAS<ADE>,CAS<ALL>,CAS<CAU>,CAS<DAT>,...,CAS<TER>,CAS<TRA>,PLUR,PLUR<FAM>,POSS,POSS<1>,POSS<1><PLUR>,POSS<2>,POSS<2><PLUR>,POSS<PLUR>
0,abbeli,abbeli,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,aberrált,aberrált,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,aberráltak,aberrált,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,aberráltakat,aberrált,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,aberráltját,aberrált,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83628,üzletivé,üzleti,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
83629,üzletszerű,üzletszerű,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
83630,üzletszerűek,üzletszerű,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
83631,üzletszerűnek,üzletszerű,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [78]:
nums = pd.read_csv('nums3.csv')

In [79]:
nums

,word,stem,morphemes
0,ahányan,ahány,NUM[AGGREG]
1,ahányszorta,ahány,NUM[MULTIPL]
2,akárhanyan,akárhány,NUM[AGGREG]
3,akárhányadszorra,akárhány,NUM[ORD-ITER-ACCOMPL]
4,akárhányan,akárhány,NUM[AGGREG]
...,...,...,...
76,öten,öt,NUM[AGGREG]
77,édeskevesen,édeskevés,NUM[AGGREG]
78,öten,öt,NUM[AGGREG]
79,ötszörte,öt,NUM[MULTIPL]


In [16]:
def one_hot_encode_num_morphemes(df):
    possible_morphemes = sorted(list(set(df['morphemes'])))
    ret = [None for i in range(len(possible_morphemes))]
    df_morphemes = list(df['morphemes'])
    for m in range(len(possible_morphemes)):
        print(m, ' / ', len(possible_morphemes))
        cur_list = [None for l in range(len(df))]
        for r, row in df.iterrows():
            if possible_morphemes[m] in row['morphemes']:
                cur_list[r] = 1
            else:
                cur_list[r] = 0
        ret[m] = cur_list
    
    for p in range(len(possible_morphemes)):
        df[possible_morphemes[p]] = ret[p]
    
    df.drop(['morphemes'], axis=1, inplace=True)
    
    return df

In [81]:
nums = one_hot_encode_num_morphemes(nums)
nums

0  /  3
1  /  3
2  /  3


,word,stem,NUM[AGGREG],NUM[MULTIPL],NUM[ORD-ITER-ACCOMPL]
0,ahányan,ahány,1,0,0
1,ahányszorta,ahány,0,1,0
2,akárhanyan,akárhány,1,0,0
3,akárhányadszorra,akárhány,0,0,1
4,akárhányan,akárhány,1,0,0
...,...,...,...,...,...
76,öten,öt,1,0,0
77,édeskevesen,édeskevés,1,0,0
78,öten,öt,1,0,0
79,ötszörte,öt,0,1,0


In [82]:
nums.to_csv('nums4.csv')

In [3]:
determiners = pd.read_csv('determiners3.csv')

In [4]:
determiners

,word,stem,ANP,ANP<PLUR>,CAS<ABL>,CAS<ACC>,CAS<ADE>,CAS<ALL>,CAS<CAU>,CAS<DAT>,...,CAS<FOR>,CAS<ILL>,CAS<INE>,CAS<INS>,CAS<SBL>,CAS<SUE>,CAS<TEM>,CAS<TRA>,PLUR,POSS<2><PLUR>
0,akként,az,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,ama,ama,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,amannak,amaz,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,amannál,amaz,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,amazok,amaz,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,ugyanazoknak,ugyanaz,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
64,ugyanazon,ugyanazon,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
65,ugyanekkor,ugyanez,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
66,ugyanezeken,ugyanez,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [5]:
list(determiners['word'])

['akként',
 'ama',
 'amannak',
 'amannál',
 'amazok',
 'amazokéi',
 'amazt',
 'amazé',
 'azokká',
 'azokéba',
 'azokéban',
 'azokéi',
 'azokéihoz',
 'azokéin',
 'azokéinál',
 'azokéit',
 'azokéitól',
 'azokéival',
 'azokén',
 'azokéra',
 'azokét',
 'azokéval',
 'azon',
 'azéért',
 'egyazon',
 'egyugyanazon',
 'eme',
 'emez',
 'emezt',
 'ezekbe',
 'ezekéinél',
 'ezekére',
 'ezekétől',
 'ezen',
 'ezetekről',
 'mindabba',
 'mindabban',
 'mindabból',
 'mindahhoz',
 'mindama',
 'mindaz',
 'mindazokba',
 'mindazokban',
 'mindazoknál',
 'mindazokra',
 'mindazokról',
 'mindazoké',
 'mindazokén',
 'mindazokéra',
 'mindazokért',
 'mindazokét',
 'mindazon',
 'mindazért',
 'mindeme',
 'mindezekben',
 'mindezekből',
 'mindezeken',
 'mindezekhez',
 'mindezekről',
 'mindezen',
 'mindezen',
 'mindezért',
 'ugyanazokban',
 'ugyanazoknak',
 'ugyanazon',
 'ugyanekkor',
 'ugyanezeken',
 'ugyanezen']

## Verbs input & output

In [17]:
chars = " 0123456789.:,;!%&'*_-=~\\()|[]{}aáäbcdeéfghiíjklmnoóöőpqrstuúüűvwxyz"

In [18]:
encode_dict = {}
decode_dict = {}

for c in range(len(chars)):
    encode_dict[chars[c]] = c
    decode_dict[c] = chars[c]

print(encode_dict)
print(decode_dict)

{' ': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, '.': 11, ':': 12, ',': 13, ';': 14, '!': 15, '%': 16, '&': 17, "'": 18, '*': 19, '_': 20, '-': 21, '=': 22, '~': 23, '\\': 24, '(': 25, ')': 26, '|': 27, '[': 28, ']': 29, '{': 30, '}': 31, 'a': 32, 'á': 33, 'ä': 34, 'b': 35, 'c': 36, 'd': 37, 'e': 38, 'é': 39, 'f': 40, 'g': 41, 'h': 42, 'i': 43, 'í': 44, 'j': 45, 'k': 46, 'l': 47, 'm': 48, 'n': 49, 'o': 50, 'ó': 51, 'ö': 52, 'ő': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'ú': 60, 'ü': 61, 'ű': 62, 'v': 63, 'w': 64, 'x': 65, 'y': 66, 'z': 67}
{0: ' ', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9', 11: '.', 12: ':', 13: ',', 14: ';', 15: '!', 16: '%', 17: '&', 18: "'", 19: '*', 20: '_', 21: '-', 22: '=', 23: '~', 24: '\\', 25: '(', 26: ')', 27: '|', 28: '[', 29: ']', 30: '{', 31: '}', 32: 'a', 33: 'á', 34: 'ä', 35: 'b', 36: 'c', 37: 'd', 38: 'e', 39: 'é', 40: 'f', 41: 'g', 42: 'h', 43: 'i', 44: 'í',

In [19]:
def encode(w):
    ret = []
    for c in w:
        ret.append(encode_dict[c])
    return np.array(ret)

In [20]:
def decode(a):
    ret = []
    for i in a:
        ret.append(decode_dict[i])
    return ''.join(ret)

In [21]:
M = len(encode_dict)
W = 44
def one_hot_encode(w):
    e = encode(w)
    ohe = np.zeros((W, M))
    ohe[np.arange(len(e)),e] = 1
    return ohe 

In [22]:
def one_hot_encode_col(col):
    return np.array([one_hot_encode(w) for w in col])

In [23]:
def stem_len(row):
    orig = str(row['word'])
    stem = str(row['stem'])
    ret = 0
    different_stem = False
    for c in range(min(len(orig),len(stem))):
        if orig[c] != stem[c]:
            ret = c
            different_stem = True
            break
    if different_stem:
        return ret
    else:
        return len(stem)

In [42]:
verbs['stem_len'] = verbs.apply(lambda row: stem_len(row), axis=1)

In [43]:
verbs

,word,stem,COND,COND<PAST>,DEF,INF,MODAL,PAST,PERS,PERS<1<OBJ<2>>>,PERS<1>,PERS<2>,PLUR,SUBJUNC-IMP,stem_len
0,abajgasd,abajgat,0,0,1,0,0,0,0,0,0,1,0,1,6
1,abajgassák,abajgat,0,0,1,0,0,0,0,0,0,0,1,1,6
2,abajgat,abajgat,0,0,0,0,0,0,0,0,0,0,0,0,7
3,abajgatja,abajgat,0,0,1,0,0,0,0,0,0,0,0,0,7
4,abajgatják,abajgat,0,0,1,0,0,0,0,0,0,0,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382350,üzérkednek,üzérkedik,0,0,0,0,0,0,0,0,0,0,1,0,7
382351,üzérkedni,üzérkedik,0,0,0,1,0,0,0,0,0,0,0,0,7
382352,üzérkedsz,üzérkedik,0,0,0,0,0,0,0,0,0,1,0,0,7
382353,üzérkedtek,üzérkedik,0,0,0,0,0,1,0,0,0,0,1,0,7


In [44]:
verbs_X = one_hot_encode_col(list(verbs['word']))
verbs_y = np.array(verbs[verbs.columns.difference(['word','stem'])])

In [45]:
verbs_X.shape

(382355, 44, 68)

In [47]:
verbs_y.shape

(382355, 13)

In [49]:
verbs_X_train, verbs_X_test, verbs_y_train, verbs_y_test = train_test_split(verbs_X, verbs_y, test_size=0.1, random_state=random_state)

In [50]:
verbs_X_train.shape, verbs_X_test.shape, verbs_y_train.shape, verbs_y_test.shape

((344119, 44, 68), (38236, 44, 68), (344119, 13), (38236, 13))

## Verbs NN

In [61]:
inp = tf.keras.Input(shape=(44,68), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(68,kernel_size=kernel_size,
        input_shape=(44,68), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model1 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(1,1,inp,'1'))
model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model1.output, model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.57')(flat)
dense12 = tf.keras.layers.Dense(12, activation='sigmoid',name='dense12')(drop)
dense_plus1 = tf.keras.layers.Dense(1, activation='relu',name='dense_plus1')(drop)

combined_out = tf.keras.layers.concatenate([dense12, dense_plus1], name='concatenate_output')

verbs_conv_model = tf.keras.Model(inputs=[model1.input],
                            outputs=combined_out,name='Verbs_Multiconv_Model')
verbs_conv_model.output_shape

(None, 13)

In [62]:
verbs_conv_model.summary()

Model: "Multiconv_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 44, 68)]     0                                            
__________________________________________________________________________________________________
1D_conv1 (Conv1D)               (None, 44, 68)       4692        input[0][0]                      
__________________________________________________________________________________________________
1D_conv3 (Conv1D)               (None, 44, 68)       13940       input[0][0]                      
__________________________________________________________________________________________________
1D_conv5 (Conv1D)               (None, 44, 68)       23188       input[0][0]                      
____________________________________________________________________________________

In [25]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3, verbose=1)

In [64]:
verbs_conv_model.compile(optimizer='adam', loss='mse')

In [65]:
verbs_conv_model.fit(verbs_X_train, verbs_y_train, callbacks=[early_stopping], epochs=20, batch_size=193, validation_split=10/193)

Train on 326289 samples, validate on 17830 samples
Epoch 1/20
326289/326289 [==============================] - 2419s 7ms/sample - loss: 0.2533 - val_loss: 0.1314
Epoch 2/20
326289/326289 [==============================] - 2355s 7ms/sample - loss: 0.1263 - val_loss: 0.1032
Epoch 3/20
326289/326289 [==============================] - 2473s 8ms/sample - loss: 0.1067 - val_loss: 0.0870
Epoch 4/20
326289/326289 [==============================] - 2300s 7ms/sample - loss: 0.0957 - val_loss: 0.0773
Epoch 5/20
326289/326289 [==============================] - 2613s 8ms/sample - loss: 0.0887 - val_loss: 0.0715
Epoch 6/20
326289/326289 [==============================] - 2510s 8ms/sample - loss: 0.0839 - val_loss: 0.0670
Epoch 7/20
326289/326289 [==============================] - 2542s 8ms/sample - loss: 0.0804 - val_loss: 0.0646
Epoch 8/20
326289/326289 [==============================] - 2291s 7ms/sample - loss: 0.0780 - val_loss: 0.0620
Epoch 9/20
326289/326289 [==============================] - 2

In [ ]:
#verbs_conv_model.fit(verbs_X_train, verbs_y_train, callbacks=[early_stopping], epochs=10, batch_size=193, validation_split=10/193)

In [66]:
verbs_conv_model.save("verbs_conv_model2.h5")

In [67]:
verbs_conv_model.evaluate(verbs_X_test, verbs_y_test)

38236/38236 [==============================] - 84s 2ms/sample - loss: 0.0551


0.05510567283219455

In [6]:
verbs_conv_model = keras.models.load_model("verbs_conv_model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [114]:
def morphemes_verb(verb):
    example = one_hot_encode_col([verb])
    prediction = verbs_conv_model.predict(example).round()
    colnames = list(verbs.columns)
    morpheme_names = colnames[2:-1]
    return verb[:int(prediction[0][-1])] + ' | ' + ' + '.join([morpheme_names[i] for i in range(len(morpheme_names)) if prediction[0][i] == 1])

In [115]:
morphemes_verb('csináltatok')

'csinált | PAST + PERS<2> + PLUR'

In [116]:
morphemes_verb('gyártsátok')

'gyárt | DEF + PERS<2> + PLUR + SUBJUNC-IMP'

In [117]:
morphemes_verb('megérthetted')

'megért | DEF + MODAL + PAST + PERS<2>'

In [118]:
morphemes_verb('megbabonázhatlak')

'megbabonáz | MODAL'

## Adjectives

In [14]:
adjectives['stem_len'] = adjectives.apply(lambda row: stem_len(row), axis=1)

In [15]:
adjectives_X = one_hot_encode_col(list(adjectives['word']))
adjectives_y = np.array(adjectives[adjectives.columns.difference(['word','stem'])])

In [16]:
adjectives_X_train, adjectives_X_test, adjectives_y_train, adjectives_y_test = train_test_split(adjectives_X, adjectives_y, test_size=0.1, random_state=random_state)

In [17]:
adjectives_X_train.shape, adjectives_X_test.shape, adjectives_y_train.shape, adjectives_y_test.shape

((75269, 44, 68), (8364, 44, 68), (75269, 29), (8364, 29))

In [ ]:
((75269, 44, 68), (8364, 44, 68), (75269, 28), (8364, 28))

In [18]:
inp = tf.keras.Input(shape=(44,68), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(68,kernel_size=kernel_size,
        input_shape=(44,68), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model1 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(1,1,inp,'1'))
model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model1.output, model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.57')(flat)
dense28 = tf.keras.layers.Dense(28, activation='sigmoid',name='dense12')(drop)
dense_plus1 = tf.keras.layers.Dense(1, activation='relu',name='dense_plus1')(drop)

combined_out = tf.keras.layers.concatenate([dense28, dense_plus1], name='concatenate_output')

adjectives_conv_model = tf.keras.Model(inputs=[model1.input],
                            outputs=combined_out,name='Adjectives_Multiconv_Model')
adjectives_conv_model.output_shape

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


(None, 29)

In [19]:
adjectives_conv_model.summary()

Model: "Adjectives_Multiconv_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 44, 68)]     0                                            
__________________________________________________________________________________________________
1D_conv1 (Conv1D)               (None, 44, 68)       4692        input[0][0]                      
__________________________________________________________________________________________________
1D_conv3 (Conv1D)               (None, 44, 68)       13940       input[0][0]                      
__________________________________________________________________________________________________
1D_conv5 (Conv1D)               (None, 44, 68)       23188       input[0][0]                      
_________________________________________________________________________

In [20]:
adjectives_conv_model.compile(optimizer='adam', loss='mse')

In [21]:
adjectives_conv_model.fit(adjectives_X_train, adjectives_y_train, callbacks=[early_stopping], epochs=10, batch_size=193, validation_split=10/193)

Train on 71369 samples, validate on 3900 samples
Epoch 1/10
71369/71369 [==============================] - 519s 7ms/sample - loss: 0.3557 - val_loss: 0.1280
Epoch 2/10
71369/71369 [==============================] - 552s 8ms/sample - loss: 0.1164 - val_loss: 0.0999
Epoch 3/10
71369/71369 [==============================] - 535s 7ms/sample - loss: 0.0990 - val_loss: 0.0886
Epoch 4/10
71369/71369 [==============================] - 529s 7ms/sample - loss: 0.0902 - val_loss: 0.0806
Epoch 5/10
71369/71369 [==============================] - 547s 8ms/sample - loss: 0.0843 - val_loss: 0.0755
Epoch 6/10
71369/71369 [==============================] - 541s 8ms/sample - loss: 0.0800 - val_loss: 0.0727
Epoch 7/10
71369/71369 [==============================] - 536s 8ms/sample - loss: 0.0763 - val_loss: 0.0677
Epoch 8/10
71369/71369 [==============================] - 572s 8ms/sample - loss: 0.0731 - val_loss: 0.0667
Epoch 9/10
71369/71369 [==============================] - 630s 9ms/sample - loss: 0.070

In [44]:
adjectives_conv_model.fit(adjectives_X_train, adjectives_y_train, callbacks=[early_stopping], epochs=10, batch_size=193, validation_split=10/193)

Train on 71369 samples, validate on 3900 samples
Epoch 1/10
71369/71369 [==============================] - 580s 8ms/sample - loss: 0.0663 - val_loss: 0.0572
Epoch 2/10
71369/71369 [==============================] - 571s 8ms/sample - loss: 0.0644 - val_loss: 0.0556
Epoch 3/10
71369/71369 [==============================] - 565s 8ms/sample - loss: 0.0626 - val_loss: 0.0543
Epoch 4/10
71369/71369 [==============================] - 527s 7ms/sample - loss: 0.0613 - val_loss: 0.0528
Epoch 5/10
71369/71369 [==============================] - 521s 7ms/sample - loss: 0.0597 - val_loss: 0.0505
Epoch 6/10
71369/71369 [==============================] - 576s 8ms/sample - loss: 0.0582 - val_loss: 0.0498
Epoch 7/10
71369/71369 [==============================] - 537s 8ms/sample - loss: 0.0575 - val_loss: 0.0482
Epoch 8/10
71369/71369 [==============================] - 534s 7ms/sample - loss: 0.0564 - val_loss: 0.0477
Epoch 9/10
71369/71369 [==============================] - 530s 7ms/sample - loss: 0.055

In [81]:
adjectives_conv_model.fit(adjectives_X_train, adjectives_y_train, callbacks=[early_stopping], epochs=10, batch_size=193, validation_split=10/193)

Train on 71369 samples, validate on 3900 samples
Epoch 1/10
71369/71369 [==============================] - 518s 7ms/sample - loss: 0.0537 - val_loss: 0.0452
Epoch 2/10
71369/71369 [==============================] - 551s 8ms/sample - loss: 0.0531 - val_loss: 0.0462
Epoch 3/10
71369/71369 [==============================] - 539s 8ms/sample - loss: 0.0526 - val_loss: 0.0439
Epoch 4/10
71369/71369 [==============================] - 530s 7ms/sample - loss: 0.0518 - val_loss: 0.0422
Epoch 5/10
71369/71369 [==============================] - 569s 8ms/sample - loss: 0.0511 - val_loss: 0.0444
Epoch 6/10
71369/71369 [==============================] - 556s 8ms/sample - loss: 0.0507 - val_loss: 0.0412
Epoch 7/10
71369/71369 [==============================] - 541s 8ms/sample - loss: 0.0502 - val_loss: 0.0410
Epoch 8/10
71369/71369 [==============================] - 517s 7ms/sample - loss: 0.0495 - val_loss: 0.0426
Epoch 9/10
71369/71369 [==============================] - 538s 8ms/sample - loss: 0.049

In [88]:
adjectives_conv_model.fit(adjectives_X_train, adjectives_y_train, callbacks=[early_stopping], epochs=10, batch_size=193, validation_split=10/193)

Train on 71369 samples, validate on 3900 samples
Epoch 1/10
71369/71369 [==============================] - 505s 7ms/sample - loss: 0.0491 - val_loss: 0.0392
Epoch 2/10
71369/71369 [==============================] - 553s 8ms/sample - loss: 0.0485 - val_loss: 0.0387
Epoch 3/10
71369/71369 [==============================] - 548s 8ms/sample - loss: 0.0479 - val_loss: 0.0397
Epoch 4/10
71369/71369 [==============================] - 546s 8ms/sample - loss: 0.0474 - val_loss: 0.0415
Epoch 5/10
71369/71369 [==============================] - 503s 7ms/sample - loss: 0.0474 - val_loss: 0.0396
Epoch 00005: early stopping


In [ ]:
adjectives_conv_model.fit(adjectives_X_train, adjectives_y_train, callbacks=[early_stopping], epochs=10, batch_size=193, validation_split=10/193)

In [90]:
adjectives_conv_model.save("adjectives_conv_model.h5")

In [89]:
adjectives_conv_model.evaluate(adjectives_X_test, adjectives_y_test)

8364/8364 [==============================] - 27s 3ms/sample - loss: 0.0431


0.043055930517396535

In [60]:
def morphemes_adjective(adjective):
    example = one_hot_encode_col([adjective])
    prediction = adjectives_conv_model.predict(example).round()
    colnames = list(adjectives.columns)
    morpheme_names = colnames[2:-1]
    return adjective[:int(prediction[0][-1])] + ' | ' + ' + '.join([morpheme_names[i] for i in range(len(morpheme_names)) if prediction[0][i] == 1])

In [91]:
morphemes_adjective('levezetőnek')

'levezető | CAS<DAT>'

In [92]:
morphemes_adjective('üzletszerűvé')

'üzletszerű | CAS<TRA>'

In [93]:
morphemes_adjective('rosszak')

'rossz | PLUR'

In [94]:
morphemes_adjective('hangosodó')

'hangosodó | '

In [101]:
morphemes_adjective('pirosakkal')

'piros | CAS<INS> + PLUR'

## Nouns

In [107]:
'''
del adjectives

'''

In [28]:
nouns['stem_len'] = nouns.apply(lambda row: stem_len(row), axis=1)

In [29]:
nouns.to_csv('nouns5.csv', index=False)

## -----------------------------------------------------------------------------------------------------------------

In [2]:
nouns = pd.read_csv('nouns5.csv')

In [3]:
nouns

,ANP,ANP<PLUR>,CAS<ABL>,CAS<ACC>,CAS<ADE>,CAS<ALL>,CAS<CAU>,CAS<DAT>,CAS<DEL>,CAS<ELA>,...,POSTP<SZÁMÁRA>,POSTP<UTÁN>,POSTP<VÉGBŐL>,POSTP<VÉGETT>,POSTP<VÉGRE>,POSTP<ÁLTAL>,POSTP<ÓTA>,stem,word,stem_len
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,aa-láv,aa-lávaként,6
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,aaaz,aaazt,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,aaaaz,aaaaz,5
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,aaah.mos,aaah.most,8
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,aaakar,aaakarok,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,üórá,üórán,4
2790638,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,üögy,üögye,4
2790639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,üögy,üögyében,4
2790640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,üönképzőkör,üönképzőkör,11


In [7]:
nouns = nouns.sample(frac=1).reset_index(drop=True)

In [8]:
nouns

,ANP,ANP<PLUR>,CAS<ABL>,CAS<ACC>,CAS<ADE>,CAS<ALL>,CAS<CAU>,CAS<DAT>,CAS<DEL>,CAS<ELA>,...,POSTP<SZÁMÁRA>,POSTP<UTÁN>,POSTP<VÉGBŐL>,POSTP<VÉGETT>,POSTP<VÉGRE>,POSTP<ÁLTAL>,POSTP<ÓTA>,stem,word,stem_len
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,sorstérkép,sorstérképet,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,tartalompartner,tartalompartnerével,15
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,perzselés,perzselésre,9
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,attaqus,attaqusnál,7
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,mérőexpedíció,mérőexpedíciónkat,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,illóolaj,illóolajokra,8
2790638,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,forma,formáét,4
2790639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,állatvédő,állatvédők,9
2790640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,programvégrehajtó,programvégrehajtók,17


In [9]:
len(nouns)

2790642

In [15]:
del nouns

In [10]:
for i in range(100):
    temp_nouns = nouns[int(len(nouns)*i/100):int(len(nouns)*(i+1)/100)].reset_index(drop=True)
    temp_nouns.to_csv('nouns5_shuffled_part_' + str(i) + '.csv', index=False)

In [11]:
inp = tf.keras.Input(shape=(44,68), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(68,kernel_size=kernel_size,
        input_shape=(44,68), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model1 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(1,1,inp,'1'))
model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model1.output, model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.57')(flat)
dense77 = tf.keras.layers.Dense(77, activation='sigmoid',name='dense12')(drop)
dense_plus1 = tf.keras.layers.Dense(1, activation='relu',name='dense_plus1')(drop)

combined_out = tf.keras.layers.concatenate([dense77, dense_plus1], name='concatenate_output')

nouns_conv_model = tf.keras.Model(inputs=[model1.input],
                            outputs=combined_out,name='Nouns_Multiconv_Model')
nouns_conv_model.output_shape

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


(None, 78)

In [12]:
nouns_conv_model.compile(optimizer='adam', loss='mse')

In [13]:
nouns_conv_model = keras.models.load_model('nouns_conv_model2.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [26]:
for e in range(5):

    for i in range(100):

        print("EPOCH " + str(e+1) + ", PHASE " + str(i+1))

        temp_nouns = pd.read_csv('nouns5_shuffled_part_' + str(i) + '.csv')

        nouns_X = one_hot_encode_col(list(temp_nouns['word']))
        nouns_y = np.array(temp_nouns[temp_nouns.columns.difference(['word','stem'])])

        nouns_X_train, nouns_X_test, nouns_y_train, nouns_y_test = train_test_split(nouns_X, nouns_y, test_size=0.1, random_state=random_state)

        nouns_conv_model.fit(nouns_X_train, nouns_y_train, callbacks=[early_stopping], epochs=1, batch_size=236, validation_split=10/236)

        nouns_conv_model.evaluate(nouns_X_test, nouns_y_test)

        del temp_nouns
        del nouns_X
        del nouns_y
        del nouns_X_train
        del nouns_X_test
        del nouns_y_train
        del nouns_y_test

EPOCH 1, PHASE 1
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0277
EPOCH 1, PHASE 2
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 18s 6ms/sample - loss: 0.0253
EPOCH 1, PHASE 3
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0259
EPOCH 1, PHASE 4
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 14s 5ms/sample - loss: 0.0266
EPOCH 1, PHASE 5
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0256
EPOCH 1, PHASE 6
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 9s 3ms/sample - loss: 0.0264
EPOCH 1, PHASE 7
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0277
EPOCH 1, PHASE 8
T

2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0259 0s - l
EPOCH 1, PHASE 70
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0252 0s - loss: 0.
EPOCH 1, PHASE 71
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 10s 4ms/sample - loss: 0.0256
EPOCH 1, PHASE 72
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0248
EPOCH 1, PHASE 73
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0252
EPOCH 1, PHASE 74
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0254
EPOCH 1, PHASE 75
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0250
EPOCH 1, PHASE 76
Train on 24050 samples, validate on 1065

2791/2791 [==============================] - 9s 3ms/sample - loss: 0.0264
EPOCH 2, PHASE 39
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0258
EPOCH 2, PHASE 40
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0251
EPOCH 2, PHASE 41
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0264
EPOCH 2, PHASE 42
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0272
EPOCH 2, PHASE 43
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 10s 4ms/sample - loss: 0.0272
EPOCH 2, PHASE 44
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0251
EPOCH 2, PHASE 45
Train on 24050 samples, validate on 1065 samples
2791/2791 [=

2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0253
EPOCH 3, PHASE 7
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0261
EPOCH 3, PHASE 8
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 15s 5ms/sample - loss: 0.0247
EPOCH 3, PHASE 9
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0243
EPOCH 3, PHASE 10
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0240
EPOCH 3, PHASE 11
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 14s 5ms/sample - loss: 0.0245
EPOCH 3, PHASE 12
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0238
EPOCH 3, PHASE 13
Train on 24050 samples, validate on 1065 samples
2791/2791 [===

2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0249
EPOCH 3, PHASE 41
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0251
EPOCH 3, PHASE 42
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 13s 5ms/sample - loss: 0.0260
EPOCH 3, PHASE 43
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0246
EPOCH 3, PHASE 44
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0240
EPOCH 3, PHASE 45
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 15s 5ms/sample - loss: 0.0245
EPOCH 3, PHASE 46
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0251
EPOCH 3, PHASE 47
Train on 24050 samples, validate on 1065 samples
2791/2791 [

2791/2791 [==============================] - 21s 7ms/sample - loss: 0.0236
EPOCH 4, PHASE 10
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0231
EPOCH 4, PHASE 11
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0245
EPOCH 4, PHASE 12
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 15s 6ms/sample - loss: 0.0238
EPOCH 4, PHASE 13
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0255
EPOCH 4, PHASE 14
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0244
EPOCH 4, PHASE 15
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 17s 6ms/sample - loss: 0.0276
EPOCH 4, PHASE 16
Train on 24050 samples, validate on 1065 samples
2791/2791 

2791/2791 [==============================] - 10s 4ms/sample - loss: 0.0244
EPOCH 4, PHASE 78
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0248
EPOCH 4, PHASE 79
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 5s 2ms/sample - loss: 0.0236
EPOCH 4, PHASE 80
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 14s 5ms/sample - loss: 0.0240
EPOCH 4, PHASE 81
Train on 24051 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0242
EPOCH 4, PHASE 82
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0236
EPOCH 4, PHASE 83
Train on 24050 samples, validate on 1065 samples
2791/2791 [==============================] - 6s 2ms/sample - loss: 0.0238 0s - loss: 0.
EPOCH 4, PHASE 84
Train on 24051 samples, validate on 1065 sampl

KeyboardInterrupt: 

In [27]:
nouns_conv_model.save("nouns_conv_model3.h5")

Teach the net further but with different data:
- remove most lines that have no morphemes (like 90% of them)
- use the already split data frame and save these differently
- remove "kutya", "nap".

In [21]:
temp_nouns = pd.read_csv('nouns5_shorter_part_0.csv')

In [13]:
print(list(temp_nouns.columns.difference(['word','stem'])))

['ANP', 'ANP<PLUR>', 'CAS<ABL>', 'CAS<ACC>', 'CAS<ADE>', 'CAS<ALL>', 'CAS<CAU>', 'CAS<DAT>', 'CAS<DEL>', 'CAS<ELA>', 'CAS<ESS>', 'CAS<FOR>', 'CAS<ILL>', 'CAS<INE>', 'CAS<INS>', 'CAS<SBL>', 'CAS<SUE>', 'CAS<TEM>', 'CAS<TER>', 'CAS<TRA>', 'PERS', 'PERS<1>', 'PERS<2>', 'PLUR', 'PLUR<ANP>', 'PLUR<FAM>', 'POSS', 'POSS<1>', 'POSS<1><PLUR>', 'POSS<2>', 'POSS<2><PLUR>', 'POSS<PLUR>', 'POSTP<ALATT>', 'POSTP<ALÁ>', 'POSTP<ALÓL>', 'POSTP<ELLEN>', 'POSTP<ELLENÉRE>', 'POSTP<ELÉ>', 'POSTP<ELÉBE>', 'POSTP<ELŐL>', 'POSTP<ELŐTT>', 'POSTP<FELETT>', 'POSTP<FELÉ>', 'POSTP<FELÜL>', 'POSTP<FELŐL>', 'POSTP<FÖLIBE>', 'POSTP<FÖLÉ>', 'POSTP<FÖLÜL>', 'POSTP<HELYETT>', 'POSTP<IRÁNT>', 'POSTP<KÖRÉ>', 'POSTP<KÖRÖTT>', 'POSTP<KÖRÜL>', 'POSTP<KÖZBEN>', 'POSTP<KÖZIBE>', 'POSTP<KÖZÉ>', 'POSTP<KÖZÖTT>', 'POSTP<KÖZÜL>', 'POSTP<LÉTÉRE>', 'POSTP<MELLETT>', 'POSTP<MELLÉ>', 'POSTP<MELLŐL>', 'POSTP<MIATT>', 'POSTP<MÖGÉ>', 'POSTP<MÖGÖTT>', 'POSTP<MÖGÜL>', 'POSTP<NÉLKÜL>', 'POSTP<RÉSZÉRE>', 'POSTP<RÉSZÉRŐL>', 'POSTP<SZERINT>', 

In [47]:
def morphemes_noun(noun):
    example = one_hot_encode_col([noun])
    prediction = nouns_conv_model.predict(example)
    print(prediction)
    rounded_prediction = prediction.round()
    colnames = list(temp_nouns.columns)
    morpheme_names = colnames[2:-1]
    return noun[:int(prediction[0][-1] + 0.5)] + ' | ' + ' + '.join([morpheme_names[i] for i in range(len(morpheme_names)) if prediction[0][i] >= 0.25])

In [54]:
morphemes_noun('napsugár')

[[2.94998586e-02 9.20206308e-04 2.64168680e-02 1.87263072e-01
  3.68008614e-02 5.63765764e-02 9.91368890e-02 3.35631669e-02
  1.84051126e-01 4.05561328e-02 2.84880400e-03 2.37868428e-02
  4.79613543e-02 3.53062451e-02 6.63159192e-02 2.61506915e-01
  1.37810916e-01 5.34012914e-03 4.28126156e-02 4.44324911e-02
  1.71735883e-03 8.90940428e-04 7.43061304e-04 1.12891525e-01
  5.85973263e-04 1.53899193e-03 2.89547116e-01 6.29415810e-02
  6.23194277e-02 3.37409079e-02 3.83728743e-03 4.94159162e-02
  5.69492579e-04 5.69194555e-04 5.70207834e-04 5.73992729e-04
  5.67615032e-04 5.76764345e-04 5.67555428e-04 5.75304031e-04
  5.75125217e-04 5.71221113e-04 5.74111938e-04 5.73724508e-04
  5.74588776e-04 5.69075346e-04 5.65618277e-04 5.76496124e-04
  5.71936369e-04 5.75751066e-04 5.66273928e-04 5.75929880e-04
  5.73635101e-04 5.64008951e-04 5.71995974e-04 5.77121973e-04
  5.75691462e-04 5.69462776e-04 5.65707684e-04 5.73188066e-04
  5.73098660e-04 5.71012497e-04 5.79565763e-04 5.69462776e-04
  5.7518

'napsug | CAS<TEM> + POSS<1><PLUR>'

## Determiners

In [14]:
determiners['stem_len'] = determiners.apply(lambda row: stem_len(row), axis=1)

In [15]:
determiners_X = one_hot_encode_col(list(determiners['word']))
determiners_y = np.array(determiners[determiners.columns.difference(['word','stem'])])

In [17]:
determiners_X_train, determiners_X_test, determiners_y_train, determiners_y_test = train_test_split(determiners_X, determiners_y, test_size=0.1, random_state=random_state)

In [18]:
determiners_X_train.shape, determiners_X_test.shape, determiners_y_train.shape, determiners_y_test.shape

((61, 44, 68), (7, 44, 68), (61, 21), (7, 21))

In [41]:
inp = tf.keras.Input(shape=(44,68), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(68,kernel_size=kernel_size,
        input_shape=(44,68), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model1 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(1,1,inp,'1'))
model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model1.output, model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.57')(flat)
dense20 = tf.keras.layers.Dense(20, activation='sigmoid',name='dense20')(drop)
dense_plus1 = tf.keras.layers.Dense(1, activation='relu',name='dense_plus1')(drop)

combined_out = tf.keras.layers.concatenate([dense20, dense_plus1], name='concatenate_output')

determiners_conv_model = tf.keras.Model(inputs=[model1.input],
                            outputs=combined_out,name='Determiners_Multiconv_Model')
determiners_conv_model.output_shape

(None, 21)

In [42]:
determiners_conv_model.compile(optimizer='adam', loss='mse')

In [43]:
determiners_conv_model.fit(determiners_X, determiners_y, callbacks=[early_stopping], epochs=1000, batch_size=10)

Epoch 1/1000
68/68 [==============================] - 2s 25ms/sample - loss: 2.1257
Epoch 2/1000
68/68 [==============================] - 0s 5ms/sample - loss: 1.4933
Epoch 3/1000
68/68 [==============================] - 0s 5ms/sample - loss: 1.0142
Epoch 4/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.7341
Epoch 5/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.6700
Epoch 6/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.5514
Epoch 7/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.4722
Epoch 8/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.4188
Epoch 9/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.3689
Epoch 10/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.3269
Epoch 11/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.2861
Epoch 12/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.2653


68/68 [==============================] - 0s 5ms/sample - loss: 0.0475
Epoch 195/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0477
Epoch 196/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0476
Epoch 197/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0480
Epoch 198/1000
68/68 [==============================] - ETA: 0s - loss: 0.045 - 0s 5ms/sample - loss: 0.0467
Epoch 199/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0476
Epoch 200/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0481
Epoch 201/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0475
Epoch 202/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0449
Epoch 203/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0465
Epoch 204/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0449
Epoch 205/1000
68/68 [==============================] - 

68/68 [==============================] - 0s 6ms/sample - loss: 0.0359
Epoch 386/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0369
Epoch 387/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0356
Epoch 388/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0369
Epoch 389/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0365
Epoch 390/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0383
Epoch 391/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0359
Epoch 392/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0355
Epoch 393/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0370
Epoch 394/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0370
Epoch 395/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0360
Epoch 396/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.

68/68 [==============================] - 0s 7ms/sample - loss: 0.0310
Epoch 578/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0285
Epoch 579/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0302
Epoch 580/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0318
Epoch 581/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0297
Epoch 582/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0319
Epoch 583/1000
68/68 [==============================] - 1s 7ms/sample - loss: 0.0312
Epoch 584/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0293
Epoch 585/1000
68/68 [==============================] - 1s 8ms/sample - loss: 0.0314
Epoch 586/1000
68/68 [==============================] - 1s 8ms/sample - loss: 0.0317
Epoch 587/1000
68/68 [==============================] - 1s 8ms/sample - loss: 0.0317
Epoch 588/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.

68/68 [==============================] - 0s 7ms/sample - loss: 0.0307
Epoch 674/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0296
Epoch 675/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0282
Epoch 676/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0271
Epoch 677/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0287
Epoch 678/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0292
Epoch 679/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0262
Epoch 680/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0294
Epoch 681/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0271
Epoch 682/1000
68/68 [==============================] - 1s 21ms/sample - loss: 0.0283
Epoch 683/1000
68/68 [==============================] - 2s 28ms/sample - loss: 0.0286
Epoch 684/1000
68/68 [==============================] - 2s 25ms/sample - loss:

68/68 [==============================] - 0s 5ms/sample - loss: 0.0259
Epoch 865/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0266
Epoch 866/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0270
Epoch 867/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0276
Epoch 868/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0277
Epoch 869/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0276
Epoch 870/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0268
Epoch 871/1000
68/68 [==============================] - 0s 6ms/sample - loss: 0.0271
Epoch 872/1000
68/68 [==============================] - 0s 5ms/sample - loss: 0.0277
Epoch 873/1000
68/68 [==============================] - 0s 7ms/sample - loss: 0.0266
Epoch 874/1000
68/68 [==============================] - 1s 8ms/sample - loss: 0.0297
Epoch 875/1000
68/68 [==============================] - 1s 8ms/sample - loss: 0.

In [44]:
determiners_conv_model.save("determiners_conv_model2.h5")

In [45]:
def morphemes_determiner(determiner):
    example = one_hot_encode_col([determiner])
    prediction = determiners_conv_model.predict(example)
    #print(prediction)
    rounded_prediction = prediction.round()
    colnames = list(determiners.columns)
    morpheme_names = colnames[2:-1]
    return determiner[:int(prediction[0][-1] + 0.5)] + ' | ' + ' + '.join([morpheme_names[i] for i in range(len(morpheme_names)) if prediction[0][i] >= 0.5])

In [29]:
print(list(determiners['word']))

['akként', 'ama', 'amannak', 'amannál', 'amazok', 'amazokéi', 'amazt', 'amazé', 'azokká', 'azokéba', 'azokéban', 'azokéi', 'azokéihoz', 'azokéin', 'azokéinál', 'azokéit', 'azokéitól', 'azokéival', 'azokén', 'azokéra', 'azokét', 'azokéval', 'azon', 'azéért', 'egyazon', 'egyugyanazon', 'eme', 'emez', 'emezt', 'ezekbe', 'ezekéinél', 'ezekére', 'ezekétől', 'ezen', 'ezetekről', 'mindabba', 'mindabban', 'mindabból', 'mindahhoz', 'mindama', 'mindaz', 'mindazokba', 'mindazokban', 'mindazoknál', 'mindazokra', 'mindazokról', 'mindazoké', 'mindazokén', 'mindazokéra', 'mindazokért', 'mindazokét', 'mindazon', 'mindazért', 'mindeme', 'mindezekben', 'mindezekből', 'mindezeken', 'mindezekhez', 'mindezekről', 'mindezen', 'mindezen', 'mindezért', 'ugyanazokban', 'ugyanazoknak', 'ugyanazon', 'ugyanekkor', 'ugyanezeken', 'ugyanezen']


In [46]:
morphemes_determiner('ekként')

'e | CAS<FOR>'

In [47]:
morphemes_determiner('amannál')

'ama | CAS<ADE>'

In [52]:
morphemes_determiner('egyugyanannál')

'egyugyana | CAS<SUE>'

In [86]:
morphemes_determiner('azzal')

'azza | '

## Nums

In [87]:
nums['stem_len'] = nums.apply(lambda row: stem_len(row), axis=1)

In [88]:
nums_X = one_hot_encode_col(list(nums['word']))
nums_y = np.array(nums[nums.columns.difference(['word','stem'])])

In [89]:
nums_X_train, nums_X_test, nums_y_train, nums_y_test = train_test_split(nums_X, nums_y, test_size=0.1, random_state=random_state)

In [90]:
nums_X_train.shape, nums_X_test.shape, nums_y_train.shape, nums_y_test.shape

((72, 44, 68), (9, 44, 68), (72, 4), (9, 4))

In [93]:
inp = tf.keras.Input(shape=(44,68), name='input')

def conv_pool_layer(kernel_size,pool_size,input_layer,name):
    
    conv = tf.keras.layers.Conv1D(68,kernel_size=kernel_size,
        input_shape=(44,68), activation='relu', padding='same',
        kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3),
        bias_regularizer=regularizers.l2(1e-3),
        activity_regularizer=regularizers.l2(1e-4),
        data_format='channels_last', name='1D_conv' + name)(input_layer)
    
    pool = max_pool3 = tf.keras.layers.MaxPooling1D(pool_size=pool_size,
        data_format='channels_first', name= '1D_maxpool' + name)(conv)

    return pool

model1 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(1,1,inp,'1'))
model3 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(3,2,inp,'3'))
model5 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(5,2,inp,'5'))
model7 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(7,4,inp,'7'))
model9 = tf.keras.Model(inputs=inp, outputs=conv_pool_layer(9,4,inp,'9'))

combined = tf.keras.layers.concatenate([model1.output, model3.output, model5.output,
                                        model7.output, model9.output], name='concatenate')

flat = tf.keras.layers.Flatten(name='flatten')(combined)
drop = tf.keras.layers.Dropout(0.57,name='droput_0.57')(flat)
dense3 = tf.keras.layers.Dense(3, activation='sigmoid',name='dense3')(drop)
dense_plus1 = tf.keras.layers.Dense(1, activation='relu',name='dense_plus1')(drop)

combined_out = tf.keras.layers.concatenate([dense3, dense_plus1], name='concatenate_output')

nums_conv_model = tf.keras.Model(inputs=[model1.input],
                            outputs=combined_out,name='Nums_Multiconv_Model')
nums_conv_model.output_shape

(None, 4)

In [94]:
nums_conv_model.compile(optimizer='adam', loss='mse')

In [95]:
nums_conv_model.fit(nums_X, nums_y, callbacks=[early_stopping], epochs=1000, batch_size=10)

Epoch 1/1000
81/81 [==============================] - 3s 40ms/sample - loss: 8.0663
Epoch 2/1000
81/81 [==============================] - 1s 8ms/sample - loss: 4.7712
Epoch 3/1000
81/81 [==============================] - 1s 8ms/sample - loss: 2.4170
Epoch 4/1000
81/81 [==============================] - 1s 8ms/sample - loss: 1.8303
Epoch 5/1000
81/81 [==============================] - 1s 8ms/sample - loss: 1.5199
Epoch 6/1000
81/81 [==============================] - 1s 7ms/sample - loss: 1.2299
Epoch 7/1000
81/81 [==============================] - 1s 8ms/sample - loss: 1.0801
Epoch 8/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.9228
Epoch 9/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.7985
Epoch 10/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.7140
Epoch 11/1000
81/81 [==============================] - 1s 9ms/sample - loss: 0.6450
Epoch 12/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.5943


81/81 [==============================] - 1s 6ms/sample - loss: 0.1703
Epoch 99/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1701
Epoch 100/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1753
Epoch 101/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1577
Epoch 102/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1688
Epoch 103/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1580
Epoch 104/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1486
Epoch 105/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1615
Epoch 106/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1571
Epoch 107/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1641
Epoch 108/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1511
Epoch 109/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1

81/81 [==============================] - 1s 8ms/sample - loss: 0.1137
Epoch 290/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1242
Epoch 291/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.1171
Epoch 292/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1129
Epoch 293/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1170
Epoch 294/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.1035
Epoch 295/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1067
Epoch 296/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1023
Epoch 297/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1100
Epoch 298/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1049
Epoch 299/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0971
Epoch 300/1000
81/81 [==============================] - 1s 9ms/sample - loss: 0.

81/81 [==============================] - 1s 7ms/sample - loss: 0.0994
Epoch 386/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1058
Epoch 387/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.0878
Epoch 388/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.1013
Epoch 389/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1043
Epoch 390/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1052
Epoch 391/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1111
Epoch 392/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.0912
Epoch 393/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0885
Epoch 394/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0952
Epoch 395/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.0958
Epoch 396/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.

81/81 [==============================] - 0s 6ms/sample - loss: 0.1161
Epoch 482/1000
81/81 [==============================] - 0s 6ms/sample - loss: 0.1141
Epoch 483/1000
81/81 [==============================] - 0s 6ms/sample - loss: 0.1030
Epoch 484/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1051
Epoch 485/1000
81/81 [==============================] - 0s 6ms/sample - loss: 0.1003
Epoch 486/1000
81/81 [==============================] - 0s 6ms/sample - loss: 0.0956
Epoch 487/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0958
Epoch 488/1000
81/81 [==============================] - 0s 6ms/sample - loss: 0.1023 0s - loss:
Epoch 489/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0945
Epoch 490/1000
81/81 [==============================] - 1s 6ms/sample - loss: 0.1008
Epoch 491/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1358
Epoch 492/1000
81/81 [==============================] - 1s 7ms/sample

81/81 [==============================] - 1s 7ms/sample - loss: 0.0817
Epoch 769/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0900
Epoch 770/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0994
Epoch 771/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.1085
Epoch 772/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0991
Epoch 773/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0983
Epoch 774/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1004
Epoch 775/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0957
Epoch 776/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0973
Epoch 777/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.1049
Epoch 778/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0992
Epoch 779/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.

81/81 [==============================] - 1s 7ms/sample - loss: 0.0843
Epoch 960/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0819
Epoch 961/1000
81/81 [==============================] - 1s 9ms/sample - loss: 0.0953
Epoch 962/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0947
Epoch 963/1000
81/81 [==============================] - 1s 8ms/sample - loss: 0.0879
Epoch 964/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0981
Epoch 965/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0797
Epoch 966/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0828
Epoch 967/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0789
Epoch 968/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0897
Epoch 969/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.0780
Epoch 970/1000
81/81 [==============================] - 1s 7ms/sample - loss: 0.

In [96]:
nums_conv_model.save("nums_conv_model.h5")

In [97]:
def morphemes_num(num):
    example = one_hot_encode_col([num])
    prediction = nums_conv_model.predict(example)
    #print(prediction)
    rounded_prediction = prediction.round()
    colnames = list(nums.columns)
    morpheme_names = colnames[2:-1]
    return num[:int(prediction[0][-1] + 0.5)] + ' | ' + ' + '.join([morpheme_names[i] for i in range(len(morpheme_names)) if prediction[0][i] >= 0.5])

In [98]:
print(list(nums['word']))

['ahányan', 'ahányszorta', 'akárhanyan', 'akárhányadszorra', 'akárhányan', 'annyian', 'annyiszorta', 'bárhányan', 'csomóan', 'egyen', 'egynéhányan', 'egypáran', 'egyszerte', 'ennyien', 'ezren', 'feleannyian', 'feleennyien', 'félannyian', 'hanyadszorra', 'hanyan', 'harmadszorra', 'harmincadszorra', 'harmincan', 'hatan', 'hatodszorra', 'hatszorta', 'hatvanadszorra', 'hatvanan', 'heten', 'hetvenen', 'huszadszorra', 'hányadszorra', 'hányan', 'hányszorta', 'háromszorta', 'hétszerte', 'hússzorta', 'húszan', 'jónéhányan', 'kevesen', 'kilencen', 'kismillióan', 'kétannyian', 'kétszerte', 'megannyian', 'milliomodszorra', 'milliárdan', 'milliárdszorta', 'millióan', 'mindahányan', 'másfelen', 'másfélszerte', 'másodszorra', 'negyvenen', 'nyolcadszorra', 'nyolcan', 'nyolcszorta', 'nyolcvanan', 'négyen', 'négyszerte', 'néhányan', 'rengetegen', 'sehányan', 'sokadszorra', 'sokan', 'számosan', 'századszorra', 'százan', 'tengernyien', 'tízen', 'tízszerte', 'ugyanannyian', 'ugyanennyien', 'valahányan', 'ö

In [103]:
morphemes_num('századszorra')

'száz | NUM[ORD-ITER-ACCOMPL]'